In [1]:
import pandas as pd

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df

,Lugar de residencia,Medida,Valor
0,Huelva,Población,510743
1,Huelva,Confirmados PDIA,11319
2,Huelva,Confirmados PDIA 14 días,646
3,Huelva,Tasa PDIA 14 días,"126,48239917140323"
4,Huelva,Confirmados PDIA 7 días,254
...,...,...,...
627,Municipio de Huelva sin especificar,Tasa PDIA 14 días,-
628,Municipio de Huelva sin especificar,Confirmados PDIA 7 días,10
629,Municipio de Huelva sin especificar,Total Confirmados,135
630,Municipio de Huelva sin especificar,Curados,115


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  11319.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  3217.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 1614 personas en los últimos 7 días 

Un positivo PCR cada 674 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,11319.0,254.0,646.0,510743.0,49.731470,126.482399,145.0
Huelva-Costa,6416.0,169.0,439.0,288115.0,58.657133,152.369713,100.0
Huelva (capital),3217.0,89.0,213.0,143663.0,61.950537,148.263645,62.0
Condado-Campiña,3470.0,62.0,173.0,155057.0,39.985296,111.571874,27.0
Sierra de Huelva-Andévalo Central,1299.0,13.0,23.0,67571.0,19.239023,34.038271,12.0
Isla Cristina,468.0,21.0,62.0,21264.0,98.758465,291.572611,11.0
Palma del Condado (La),555.0,16.0,36.0,10761.0,148.685066,334.541399,8.0
Villablanca,66.0,10.0,11.0,2848.0,351.123596,386.235955,7.0
Cerro de Andévalo (El),45.0,7.0,8.0,2364.0,296.108291,338.409475,6.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Villablanca,66.0,10.0,11.0,2848.0,351.123596,386.235955,7.0
Lucena del Puerto,106.0,2.0,13.0,3371.0,59.329576,385.642243,1.0
Cerro de Andévalo (El),45.0,7.0,8.0,2364.0,296.108291,338.409475,6.0
Palma del Condado (La),555.0,16.0,36.0,10761.0,148.685066,334.541399,8.0
Manzanilla,52.0,7.0,7.0,2135.0,327.868852,327.868852,NaN
Bonares,67.0,3.0,19.0,6058.0,49.521294,313.634863,0.0
Isla Cristina,468.0,21.0,62.0,21264.0,98.758465,291.572611,11.0
Cartaya,430.0,17.0,58.0,19974.0,85.110644,290.377491,4.0
Palos de la Frontera,261.0,10.0,32.0,11289.0,88.581805,283.461777,4.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Lucena del Puerto,106.0,2.0,13.0,3371.0,59.329576,385.642243,1.0,0.153846
Bonares,67.0,3.0,19.0,6058.0,49.521294,313.634863,0.0,0.157895
Aljaraque,443.0,5.0,26.0,21260.0,23.518344,122.295390,3.0,0.192308
Rociana del Condado,121.0,1.0,5.0,7855.0,12.730745,63.653724,0.0,0.200000
Cartaya,430.0,17.0,58.0,19974.0,85.110644,290.377491,4.0,0.293103
Palos de la Frontera,261.0,10.0,32.0,11289.0,88.581805,283.461777,4.0,0.312500
Trigueros,134.0,1.0,3.0,7713.0,12.965124,38.895371,0.0,0.333333
Isla Cristina,468.0,21.0,62.0,21264.0,98.758465,291.572611,11.0,0.338710
Condado-Campiña,3470.0,62.0,173.0,155057.0,39.985296,111.571874,27.0,0.358382
